In [6]:
from datetime import timedelta, datetime
from django.conf import settings
import os
import glob
from time import mktime
import logging
from apps.photo.models import upload_image_to
logger = logging.getLogger(__name__)

def timestamp(delta, fallback):
    """Calculates seconds since epoch of current time minus delta.
    If delta is out of range, calculates timestamp of fallback datetime instead.
    """
    try:
        dt = datetime.now() - delta
    except (OverflowError):
        # logger.exception('locals:%s' % locals())
        dt = fallback
    return int(mktime(dt.timetuple()))

def new_staging_files(staging_subdirectory, fileglob='*.*', min_age=timedelta.min, max_age=timedelta.max):
    """Check for new or updated files in staging area."""
    directory = os.path.join(settings.STAGING_ROOT, staging_subdirectory)
    os.chdir(directory)
    all_files = glob.glob(fileglob)
    min_mtime = timestamp(min_age, fallback=datetime.max)
    max_mtime = timestamp(max_age, fallback=datetime.min)
    files = [file for file in all_files 
             if min_mtime > os.path.getmtime(file) > max_mtime]
    return directory, sorted(files)

def new_staging_images(**kwargs):
    """Check for new or updated images in staging area"""
    arguments = dict(
        staging_subdirectory='IMAGES',
        fileglob='*.jpg',
    )
    arguments.update(kwargs)
    return new_staging_files(**arguments)

def new_staging_byline_images(**kwargs):
    """Check for new or updated byline images in staging area"""
    arguments = dict(
        staging_subdirectory='BYLINE',
        fileglob='*.jpg',
        max_age=timedelta(days=1),
    )
    arguments.update(kwargs)
    return new_staging_files(**arguments)

def new_staging_pdf_files(**kwargs):
    """Check for new or updated pdf files in staging area"""
    arguments = dict(
        staging_subdirectory='PDF',
        fileglob='UNI1*VER*000.pdf',
        max_age=timedelta(days=100),from apps.photo.models import upload_image_to
    )
    arguments.update(kwargs)
    return new_staging_files(**arguments)


In [28]:
def import_staging_images():
    directory, files = new_staging_images(max_age=timedelta(days=7))
    for file in files:
        full_path = os.path.join(directory, file)
        print(full_path)
        dest = upload_image_to(ImageFile, file)
        try:
            try:
                img = ImageFile.objects.get(
                    source_file__endswith='/' + os.path.basename(dest))
            except ImageFile.MultipleObjectsReturned:
                img = ImageFile.objects.get(source_file=dest)
        except ImageFile.DoesNotExist:
            img = ImageFile()
        img.save_local_image_as_source(full_path)
        print(img.source_file)



/srv/fotoarkiv_universitas/STAGING/IMAGES/22-NYH-trangtpajuss-01-SBV.jpg
2015/30/22-NYH-trangtpajuss-01-SBV.jpg
/srv/fotoarkiv_universitas/STAGING/IMAGES/25-NYH-nyhsiobolige-01-HJK.jpg
2015/30/25-NYH-nyhsiobolige-01-HJK.jpg
/srv/fotoarkiv_universitas/STAGING/IMAGES/25-NYH-nyhsiobolige-03-HJK.jpg
2015/30/25-NYH-nyhsiobolige-03-HJK.jpg
/srv/fotoarkiv_universitas/STAGING/IMAGES/25-NYH-nyhsiobolige-04-HJK.jpg
2015/30/25-NYH-nyhsiobolige-04-HJK.jpg
/srv/fotoarkiv_universitas/STAGING/IMAGES/2527792696-db31f5aaf4-b.jpg
2015/30/2527792696-db31f5aaf4-b.jpg
/srv/fotoarkiv_universitas/STAGING/IMAGES/28-KUL-minstudietid-01-OJ.jpg
2015/30/28-KUL-minstudietid-01-OJ.jpg
/srv/fotoarkiv_universitas/STAGING/IMAGES/28-KUL-minstudietid-02-OJ.jpg
2015/30/28-KUL-minstudietid-02-OJ.jpg
/srv/fotoarkiv_universitas/STAGING/IMAGES/28-KUL-minstudietid-03-OJ.jpg
2015/30/28-KUL-minstudietid-03-OJ.jpg
/srv/fotoarkiv_universitas/STAGING/IMAGES/28-NYH-karakterkaos-10-KEK.jpg
2015/30/28-NYH-karakterkaos-10-KEK.jpg
/srv

In [20]:
%debug

> /srv/local.universitas.no/pyvenv/local.universitas.no/lib/python3.4/site-packages/django/db/models/query.py(338)get()
    337             "get() returned more than one %s -- it returned %s!" %
--> 338             (self.model._meta.object_name, num)
    339         )

ipdb> u
> <ipython-input-19-c34034d7328e>(14)<module>()
     13     else:
---> 14         img = saved.get()
     15         img.save_local_image_as_source(full_path)

ipdb> saved
[<ImageFile: 1.jpg>, <ImageFile: 1.jpg>]
ipdb> saved[0].md5
'51bc5b1ea8ffd527abc83d3a58836513'
ipdb> saved[1].md5
'062e359817396b533d0e7e4d12fde3b6'
ipdb> q


In [47]:
def size(fname):
    return int(os.stat(fname).st_size)

def mtime(fname):
    return datetime.fromtimestamp(os.stat(fname).st_mtime)

def ctime(fname):
    return datetime.fromtimestamp(os.stat(fname).st_ctime)

def hashfile(fname, algorithm=hashlib.md5, blocksize=65536):
    """Hexadecimal hash of a file"""
    hasher = algorithm()
    with open(fname, 'rb') as source:
        buffer = source.read(blocksize)
        while len(buffer) > 0:
            hasher.update(buffer)
            buffer = source.read(blocksize)
    return hasher.hexdigest()

def stats(file):
    print('filename: {file}\nmtime:    {mtime}\nctime:    {ctime}\nmd5:      {md5}'.format(
        file=file, mtime=mtime(file), ctime=ctime(file), size=size(file), md5=hashfile(file)))

stats(file)

filename: 21-DEB-debbrei-12-PSS.jpg
mtime:    2015-08-21 15:27:05
ctime:    2015-10-16 23:24:37.027950
md5:      98e89e5dc7d328dfcbb2d69633195d05


In [5]:
{datetime.now():'now'}

{datetime.datetime(2015, 10, 22, 20, 47, 9, 422159): 'now'}

In [6]:
datetime.min.strftime('%s')

'-62135599380'

In [9]:
import os
from datetime import datetime

now = datetime.now()
IMAGE_STAGING = os.path.join(settings.STAGING_ROOT, 'IMAGES')
os.chdir(IMAGE_STAGING)
print(os.path.abspath(os.curdir))

/srv/fotoarkiv_universitas/STAGING/IMAGES


In [11]:
for file_path in sorted(os.listdir(), key=lambda f: os.path.getmtime(f), reverse=True)[:10]: 
    mtime = os.path.getmtime(file_path)
    modified = datetime.fromtimestamp(mtime)
    print('{0:<40.40}  {1}'.format(file_path, modified))
    

Nummer-14-onsdag-18-oktober1989.jpg       2015-10-14 11:32:00
28-SID-oyeblikk-03-AOB.jpg                2015-10-14 11:31:58
statsbudsjett.jpg                         2015-10-14 11:31:58
28-SID-oyeblikk-02-AOB.jpg                2015-10-14 11:31:57
28-SID-oyeblikk-01-AOB.jpg                2015-10-14 11:31:56
28-NYH-nyhrektorkje-03-XX.jpg             2015-10-14 11:31:54
rektor plektor fra _SLEKTOR_.JPEG         2015-10-14 11:31:54
Arkivfoto-uio-rektor-KjetilBlom.jpg       2015-10-14 11:31:54
28-NYH-nyhrektorkje-01-XX.jpg             2015-10-14 11:31:52
26-NYH-nyharbeidsmo-03-HB.jpg             2015-10-14 11:31:52


In [12]:
ImageFile.objects.filter(source_file__icontains=file_path)

[]

In [13]:
file_path

'26-NYH-nyharbeidsmo-03-HB.jpg'

In [17]:
ImageFile.objects.filter(source_file__icontains='26-NYH-nyha')

[]

In [53]:
ImageFile.objects.filter(source_file__size__gt=10**6)

FieldError: Unsupported lookup 'size' for ImageField or join on the field not permitted.